In [ ]:
import geopandas
import matplotlib.pyplot as plt
import momepy
import numpy as np
import osmnx
import pandas as pd
import sklearn.cluster
import shapely.ops

from pandas import CategoricalDtype

# Corridor Segments

In [ ]:
city_name = "Bucharest"
river_name = "Dâmbovița"

In [ ]:
edges = geopandas.read_file(f"../data/generated/street_network_edges_{city_name}.gpkg")
nodes = geopandas.read_file(f"../data/generated/street_network_nodes_{city_name}.gpkg")
waterway = geopandas.read_file(f"../data/generated/waterway_{river_name}.gpkg")
corridor = geopandas.read_file(f"../data/generated/corridor_{river_name}.gpkg")


## Cleaning with OSMnx

Ultimately not superior to what can be done with sfnetworks. Also, does not seem to work smoothly everywhere. Skipping it for now.

In [ ]:
nodes['x'] = nodes.geometry.x
nodes['y'] = nodes.geometry.y
nodes = nodes.rename_axis('osmid')

In [ ]:
edges['key'] = 0
edges['osmid'] = edges.index
edges = edges.rename({'from': 'u', 'to': 'v'}, axis='columns')
edges = edges.set_index(['u', 'v', 'key'])

In [ ]:
g = osmnx.graph_from_gdfs(nodes, edges)

In [ ]:
# this fails, who knows why
# g_consolidated = osmnx.consolidate_intersections(g, rebuild_graph=True, tolerance=15, dead_ends=False)

Reproducing example notebook seems to work..

In [ ]:
point = 37.858495, -122.267468
G = osmnx.graph_from_point(point, network_type="drive", dist=500)
G_proj = osmnx.project_graph(G)
n, e = osmnx.graph_to_gdfs(G_proj)
r = osmnx.graph_from_gdfs(n, e)

In [ ]:
G2 = osmnx.consolidate_intersections(r, rebuild_graph=True, tolerance=20, dead_ends=False)

## Continuity analysis

In [ ]:
# [i for i in range(len(continuity.stroke_gdf())) if i not in continuity.stroke_attribute().unique()]

In [ ]:
# strokes.reset_index().explore(column="stroke_group", cmap="prism")

In [ ]:
# convert highway level to category - it makes it easier to merge attributes for the strokes
cat = CategoricalDtype(categories=['motorway', 'primary', 'secondary', 'tertiary'], ordered=True)
edges["highway"] = edges["highway"].astype(cat)

In [ ]:
# make continuity analysis using COINS
continuity = momepy.COINS(edges)
# strokes = continuity.stroke_gdf()  # this somehow differ from what we get from the procedure below ...
edges["stroke_group"] = continuity.stroke_attribute()
strokes = edges.dissolve(by="stroke_group", aggfunc="min")


In [ ]:
# plotting strokes up to primary, and comparing it with the original level attributes
m = corridor.explore()
m = edges[edges["highway"] == "secondary"].reset_index().explore(m=m, color="black")
m = edges[edges["highway"] <= "primary"].reset_index().explore(m=m, color="red")
m = strokes[strokes["highway"] <= "primary"].reset_index().explore(m=m, color="green")
m

## Splitting the corridor into blocks

In [ ]:
def filter_edges(edges, geometry, max_level="primary"):
    """
    Select edges that intersect the corridor, up to a
    specified level for 'highway'
    """
    filtered = edges[edges["highway"] <= max_level]
    return filtered[filtered.intersects(geometry)]

In [ ]:
def get_blocks(edges: geopandas.GeoSeries, corridor: shapely.Polygon):
    """
    Clip `corridor_geom` into blocks using LineStrings
    from `edge_geoms` (both as geopandas.GeoSeries)
    """
    lines = edges.to_list()
    lines.append(corridor.boundary)
    lines_merged = shapely.ops.linemerge(lines)
    border_lines = shapely.ops.unary_union(lines_merged)
    decomposition = shapely.ops.polygonize(border_lines)

    # decomposition can extend beyond the corridor - clip it now
    decomposition_gdf = geopandas.GeoSeries(decomposition, crs=edges.crs)
    blocks = decomposition_gdf.clip(corridor)
    return blocks[blocks.type == "Polygon"]  # drop elements at the edges (LineStrings and Points)

In [ ]:
# using the strokes to break corridor into blocks
corridor_geom = corridor.iloc[0].geometry
strokes_intersecting = filter_edges(strokes, corridor_geom)
blocks_strokes = get_blocks(
    strokes_intersecting.explode().geometry, # split multilinestrings
    corridor_geom
)

In [ ]:
blocks_strokes.explore()

In [ ]:
# using the original network to break corridor into blocks
edges_intersecting = filter_edges(edges, corridor_geom)
blocks_edges = get_blocks(
    edges_intersecting.explode().geometry, # split multilinestrings
    corridor_geom
)

In [ ]:
blocks_edges.explore()

## Finding the midpoints of the river segments

These are expected to represent the "centroids" of the segments. 

In [ ]:
# define the river geometry and the street network of interest
waterway_geom = waterway.iloc[0].geometry
streets = edges[edges['highway'] <= "primary"]


In [ ]:
# find the crossings as intersections between river and  the street network
geoms = streets.intersection(waterway_geom)
crossings = geoms[~geoms.geometry.is_empty]  # this should now be "point" geometries

In [ ]:
# group the crossings in clusters and dissolve the grouped points into centroids
xy = np.column_stack([crossings.x, crossings.y])
dbscan = sklearn.cluster.DBSCAN(eps=100, min_samples=1)
dbscan.fit(xy)
crossings_clustered = crossings.to_frame("geometry")
crossings_clustered['cluster'] = dbscan.labels_
crossings_dissolved = crossings_clustered.dissolve(by="cluster").centroid

In [ ]:
# find distances of (clustered) crossings along the waterway and sort them
dists = shapely.line_locate_point(waterway_geom, crossings_dissolved)

In [ ]:
# identify midpoints between (clustered) crossings: these distances are used to find the centroids of the segments!
dist_endpoints = [0., *sorted(dists), waterway_geom.length]
dist_centroids = list(map(
    lambda x: sum(x)/len(x),
    zip(dist_endpoints[1:], dist_endpoints[:-1])
))
centroids = waterway_geom.interpolate(dist_centroids)
centroids = geopandas.GeoDataFrame(geometry=centroids, crs=waterway.crs)


## Merging the blocks into the river segments

### 1. Using the blocks from the strokes

In [ ]:
# attempt 1.1: merging the blocks that are closest to the centroids of the segments
blocks_grouped = blocks_strokes.to_frame("geometry").sjoin_nearest(centroids)
segments = blocks_grouped.dissolve(by="index_right").reset_index()

In [ ]:
segments.explore(column="index_right", categorical=True)

In [ ]:
# attempt 1.2: merging the blocks whose centroids are closest to centroids of the segments
blocks_grouped = blocks_strokes.to_frame('block')
blocks_grouped['centroid'] = blocks_grouped.centroid
blocks_grouped = blocks_grouped \
    .set_geometry('centroid') \
    .sjoin_nearest(centroids) \
    .set_geometry('block')
segments = blocks_grouped.dissolve(by="index_right").reset_index()

In [ ]:
segments.explore(column="index_right", categorical=True)

### 2. Using the blocks from the original network edges

In [ ]:
# attempt 2.1: merging the blocks that are closest to the centroids of the segments
blocks_grouped = blocks_edges.to_frame("geometry").sjoin_nearest(centroids)
segments = blocks_grouped.dissolve(by="index_right").reset_index()

In [ ]:
segments.explore(column="index_right", categorical=True)

In [ ]:
# attempt 2.2: merging the blocks whose centroids are closest to centroids of the segments
blocks_grouped = blocks_edges.to_frame('block')
blocks_grouped['centroid'] = blocks_grouped.centroid
blocks_grouped = blocks_grouped \
    .set_geometry('centroid') \
    .sjoin_nearest(centroids) \
    .set_geometry('block')
segments = blocks_grouped.dissolve(by="index_right").reset_index()

In [ ]:
segments.explore(column="index_right", categorical=True)